# Hierarchical Indexing (Think Panel Data)
#### Higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In [4]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series
#### Let's start by considering how we might represent two-dimensional data within a one-dimensional Series.

### The bad way
#### Suppose you would like to track data about states from two different years. Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [5]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

#### With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [6]:
pop[('California', 2010): ('Texas', 2000)] #select from cali 2010 to texas 2000

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

#### But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [7]:
pop[[i for i in pop.index if i[1] ==2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

In [8]:
pop[[1]]

(California, 2010)    37253956
dtype: int64

In [9]:
pop[0]

33871648

In [10]:
pop[[0]]

(California, 2000)    33871648
dtype: int64

#### This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

## The Better Way: Pandas MultIndex

#### Fortunately, Pandas provides a better way. Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas MultiIndex type gives us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

In [11]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

#### Notice that the MultiIndex contains multiple levels of indexing–in this case, the state names and the years, as well as multiple labels for each data point which encode these levels. 

#### If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:

In [12]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [13]:
pop[:, 2010] #all data for 2010

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [14]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

## MultiIndex as extra dimension

#### You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and column labels. In fact, Pandas is built with this equivalence in mind. The unstack() method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:

In [15]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [16]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


#### Seeing this, you might wonder why would we would bother with hierarchical indexing at all. The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional Series, we can also use it to represent data of three or more dimensions in a Series or DataFrame. Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a MultiIndex this is as easy as adding another column to the DataFrame:

In [17]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

#### In addition, all the ufuncs and other functionality discussed in Operating on Data in Pandas work with hierarchical indices as well. Here we compute the fraction of people under 18 by year, given the above data:

In [18]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

#### The most straightforward way to construct a multiply indexed Series or DataFrame is to simply pass a list of two or more index arrays to the constructor. For example:

In [19]:
df = pd.DataFrame(np.random.rand(4,2),
                 index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.906037  0.924286
  2  0.158807  0.446501
b 1  0.937946  0.337442
  2  0.742898  0.612617

#### The work of creating the MultiIndex is done in the background.Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a MultiIndex by default:

In [20]:

data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Nevertheless, it is sometimes useful to explicitly create a MultiIndex; we'll see a couple of these methods here.

## Explicit MultiIndex constructors

#### For more flexibility in how the index is constructed, you can instead use the class method constructors available in the pd.MultiIndex. For example, as we did before, you can construct the MultiIndex from a simple list of arrays giving the index values within each level:

pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]])

#### You can construct it from a list of tuples giving the multiple index values of each point:

In [21]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [22]:
#### You can even construct it from a Cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

#### Similarly, you can construct the MultiIndex directly using its internal encoding by passing levels (a list of lists containing available index values for each level) and labels (a list of lists that reference these labels):

In [23]:
example = pd.MultiIndex(levels = [['a', 'b'], [1,2]],
              labels = [[0, 0, 1, 1], [0, 1, 0, 1]])

#### Any of these objects can be passed as the index argument when creating a Series or Dataframe, or be passed to the reindex method of an existing Series or DataFrame.

In [24]:
example[[3]]

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[1], [1]])

In [25]:
print(example)

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])


## Naming  MultiIndex levels

#### Sometimes it is convenient to name the levels of the MultiIndex. This can be accomplished by passing the names argument to any of the above MultiIndex constructors, or by setting the names attribute of the index after the fact:

In [26]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [27]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

In [28]:
index

MultiIndex(levels=[[2013, 2014], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['year', 'visit'])

In [29]:
columns

MultiIndex(levels=[['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['subject', 'type'])

In [30]:
##mock some data
data = np.round(np.random.randn(4,6),1)
data

array([[ 0.2,  0.2,  0.9,  2. ,  0.3,  1. ],
       [-0.1,  0.5, -0.4,  0. ,  0.2,  1. ],
       [-0.8, -0.3,  0.2, -1.3, -0.9,  0.7],
       [ 1. , -1.3,  1.7,  1.4,  1.1, -0.4]])

In [31]:
data[:, ::2] *= 10

In [32]:
data[:, ::2]

array([[ 2.,  9.,  3.],
       [-1., -4.,  2.],
       [-8.,  2., -9.],
       [10., 17., 11.]])

In [33]:
data += 37 #add 37 to every value in array

In [34]:
data

array([[39. , 37.2, 46. , 39. , 40. , 38. ],
       [36. , 37.5, 33. , 37. , 39. , 38. ],
       [29. , 36.7, 39. , 35.7, 28. , 37.7],
       [47. , 35.7, 54. , 38.4, 48. , 36.6]])

In [35]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.2  46.0  39.0  40.0  38.0
     2      36.0  37.5  33.0  37.0  39.0  38.0
2014 1      29.0  36.7  39.0  35.7  28.0  37.7
     2      47.0  35.7  54.0  38.4  48.0  36.6

#### Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, for example, index the top-level column by the person's name and get a full DataFrame containing just that person's information:

In [36]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      46.0  39.0
     2      33.0  37.0
2014 1      39.0  35.7
     2      54.0  38.4

In [37]:
#### For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

In [38]:
health_data.unstack()

subject   Bob                   Guido                     Sue              \
type       HR        Temp          HR        Temp          HR        Temp   
visit       1     2     1     2     1     2     1     2     1     2     1   
year                                                                        
2013     39.0  36.0  37.2  37.5  46.0  33.0  39.0  37.0  40.0  39.0  38.0   
2014     29.0  47.0  36.7  35.7  39.0  54.0  35.7  38.4  28.0  48.0  37.7   

subject        
type           
visit       2  
year           
2013     38.0  
2014     36.6

## Indexing and Slicing a MultiIndex

#### Indexing and slicing on a MultiIndex is designed to be intuitive, and it helps if you think about the indices as added dimensions. We'll first look at indexing multiply indexed Series, and then multiply-indexed DataFrames.

### Multiply indexed Series

In [39]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [40]:
#We can access single elements by indexing with multiple terms:
pop['California', 2000]

33871648

In [41]:
# The MultiIndex also supports partial indexing, or indexing just one 
#of the levels in the index. The result is another Series, with the 
#lower-level indices maintained:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [42]:
#Partial slicing is available as well, as long as the MultiIndex 
#is sorted (see discussion in Sorted and Unsorted Indices):
pop.loc['California': 'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [43]:
#With sorted indices, partial indexing can be performed on lower 
#levels by passing an empty slice in the first index:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [44]:
#Other types of indexing and selection 
#(discussed in Data Indexing and Selection) work as well; 
#for example, selection based on Boolean masks:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [45]:
#Selection based on fancy indexing also works:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [46]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [47]:
pop[0]

33871648

In [48]:
pop[[0]]

state       year
California  2000    33871648
dtype: int64

In [49]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [50]:
pop[['Texas']]

state  year
Texas  2000    20851820
       2010    25145561
dtype: int64

In [51]:
pop[:, 2010]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

## Multiply Indexed DataFrames

In [52]:
#A multiply indexed DataFrame behaves in a similar manner. 
#Consider our toy medical DataFrame from before:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.2  46.0  39.0  40.0  38.0
     2      36.0  37.5  33.0  37.0  39.0  38.0
2014 1      29.0  36.7  39.0  35.7  28.0  37.7
     2      47.0  35.7  54.0  38.4  48.0  36.6

In [53]:
#A multiply indexed DataFrame behaves in a similar manner. 
#Consider our toy medical DataFrame from before:
health_data['Guido', 'HR']

year  visit
2013  1        46.0
      2        33.0
2014  1        39.0
      2        54.0
Name: (Guido, HR), dtype: float64

In [54]:
#Also, as with the single-index case, we can use the loc, iloc, 
#and ix indexers introduced in Data Indexing and Selection. For example:

health_data.iloc[:2, :2] #first two rows, first two columns

subject      Bob      
type          HR  Temp
year visit            
2013 1      39.0  37.2
     2      36.0  37.5

In [55]:
health_data.iloc[:2] # first two rows

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.2  46.0  39.0  40.0  38.0
     2      36.0  37.5  33.0  37.0  39.0  38.0

In [56]:
health_data.iloc[2:, :2] #last two rows, first two columns

subject      Bob      
type          HR  Temp
year visit            
2014 1      29.0  36.7
     2      47.0  35.7

In [57]:
#These indexers provide an array-like view of the underlying 
#two-dimensional data, but each individual index in loc or iloc 
#can be passed a tuple of multiple indices. For example:

health_data.loc[:, ('Bob', 'HR')] #all rows, only bob & hr column

year  visit
2013  1        39.0
      2        36.0
2014  1        29.0
      2        47.0
Name: (Bob, HR), dtype: float64

In [58]:
#Working with slices within these index tuples is not 
#especially convenient; trying to create a slice within 
#a tuple will lead to a syntax error:

health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-58-1707be15411a>, line 5)

In [59]:
#You could get around this by building the desired slice 
#explicitly using Python's built-in slice() function, but a better way 
#in this context is to use an IndexSlice object, which Pandas provides 
#for precisely this situation. For example:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']] #first row, HR Column

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,39.0,46.0,40.0
2014,1,29.0,39.0,28.0


## Rearranging Multi-Indices

In [60]:
# One of the keys to working with multiply indexed data is knowing 
#how to effectively transform the data. There are a number of operations 
#that will preserve all the information in the dataset, but rearrange it 
#for the purposes of various computations. We saw a brief example of this 
#in the stack() and unstack() methods, but there are many more ways to finely 
#control the rearrangement of data between hierarchical indices and columns, 
#and we'll explore them here.

#### Sorted and unsorted indices

In [61]:
#Earlier, we briefly mentioned a caveat, but we should emphasize it more here. 
#Many of the MultiIndex slicing operations will fail if the index is not sorted. 
#Let's take a look at this here.

#We'll start by creating some simple multiply indexed data where the 
#indices are not lexographically sorted:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.381337
      2      0.176098
c     1      0.818413
      2      0.896978
b     1      0.658973
      2      0.407774
dtype: float64

In [62]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [63]:
data = data.sort_index()
data

char  int
a     1      0.381337
      2      0.176098
b     1      0.658973
      2      0.407774
c     1      0.818413
      2      0.896978
dtype: float64

In [64]:
#With the index sorted in this way, partial slicing will work as expected:

data['a':'b']

char  int
a     1      0.381337
      2      0.176098
b     1      0.658973
      2      0.407774
dtype: float64

#### Stacking and Unstacking indices

In [65]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [66]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [67]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [68]:
# The opposite of unstack() is stack(), which here can be used to 
#recover the original series:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Index setting and resetting

In [74]:
#Another way to rearrange hierarchical data is to turn the index labels into 
#columns; this can be accomplished with the reset_index method. 
#Calling this on the population dictionary will result in a DataFrame 
#with a state and year column holding the information that was formerly 
#in the index. For clarity, we can optionally specify the name of the data 
#for the column representation:

pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [70]:
#Often when working with data in the real world, the raw input data looks 
#like this and it's useful to build a MultiIndex from the column values. 
#This can be done with the set_index method of the DataFrame, which 
#returns a multiply indexed DataFrame:

pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [71]:
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [72]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [75]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.2  46.0  39.0  40.0  38.0
     2      36.0  37.5  33.0  37.0  39.0  38.0
2014 1      29.0  36.7  39.0  35.7  28.0  37.7
     2      47.0  35.7  54.0  38.4  48.0  36.6

In [76]:
#Perhaps we'd like to average-out the measurements in the two visits 
#each year. We can do this by naming the index level we'd like to 
#explore, in this case the year

data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     37.5  37.35  39.5  38.00  39.5  38.00
2014     38.0  36.20  46.5  37.05  38.0  37.15

In [77]:
#By further making use of the axis keyword, we can take the mean among 
#levels on the columns as well:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,38.833333,37.783333
2014,40.833333,36.800000


In [ ]:
#Thus in two lines, we've been able to find the average heart rate 
#and temperature measured among all subjects in all visits each year. 
#This syntax is actually a short cut to the GroupBy functionality, 
#which we will discuss in Aggregation and Grouping. While this is a 
#toy example, many real-world datasets have similar hierarchical structure.